In [1]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
# read weather csv file into dataframe
weather_file = "../Resources/weather_data_nyc_centralpark_2016.csv"
df_weather_data = pd.read_csv(weather_file)
df_weather_data.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


In [3]:
# read sales csv file into dataframe
sales_file = "../Resources/Sales_2016.csv"
df_sales_data = pd.read_csv(sales_file)
df_sales_data.head()

,DATE,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,TOTAL_RETURN_SALES,TOTAL_RETURN_UNITS
0,1/1/2016,S,1604,101482.59,8046,769,0.00,0
1,1/1/2016,R,107,0.00,0,0,7631.56,169
2,1/2/2016,S,2669,174602.55,13305,1446,0.00,0
3,1/2/2016,R,214,0.00,0,0,13213.79,359
4,1/3/2016,S,2298,152357.03,11795,1324,0.00,0


### Transform weather DataFrame

In [4]:
# replace string 'T's in data with null values
df_weather_data["precipitation"] = df_weather_data["precipitation"].replace('T', np.nan, regex=True)
df_weather_data["snow fall"] = df_weather_data["snow fall"].replace('T', np.nan, regex=True)
df_weather_data["snow depth"] = df_weather_data["snow depth"].replace('T', np.nan, regex=True)

# define function to convert dates since dates in raw data are in 2 different formats
def try_parsing_date(text):
    for fmt in ('%m/%d/%Y', '%d-%m-%Y'):
        try:
            return datetime.strptime(text, fmt).date()
        except ValueError:
            pass
    raise ValueError('no valid date format found')

# convert dates
i = 0
for date in df_weather_data["date"]:
    df_weather_data["date"][i] = try_parsing_date(date)
    i += 1

# rename column headers
df_weather_data = df_weather_data.rename(columns={"maximum temperature": "max_temp",
                                                  "minimum temperature": "min_temp",
                                                  "average temperature": "avg_temp",
                                                  "snow fall": "snow_fall",
                                                  "snow depth": "snow_depth"})

# convert object types to floats
df_weather_data["precipitation"] = df_weather_data.precipitation.astype(float)
df_weather_data["snow_fall"] = df_weather_data.snow_fall.astype(float)
df_weather_data["snow_depth"] = df_weather_data.snow_depth.astype(float)

df_weather_data.head()

C:\Users\Allen\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,max_temp,min_temp,avg_temp,precipitation,snow_fall,snow_depth
0,2016-01-01,42,34,38.0,0.0,0.0,0.0
1,2016-01-02,40,32,36.0,0.0,0.0,0.0
2,2016-01-03,45,35,40.0,0.0,0.0,0.0
3,2016-01-04,36,14,25.0,0.0,0.0,0.0
4,2016-01-05,29,11,20.0,0.0,0.0,0.0


In [5]:
# write df to csv
df_weather_data.to_csv("../Resources/Cleaned_Weather_Data.csv", index=False)

### Transform sales DataFrame

In [6]:
# split Sales and Returns data from Sales file
sale_only_df = df_sales_data[df_sales_data["TRANSACTION_TYPE"] == "S"]
return_only_df = df_sales_data[df_sales_data["TRANSACTION_TYPE"] == "R"]

# select columns needed
sales_weather_cols = ["DATE", "TRANSACTION_TYPE", "TOTAL_TRANSACTIONS", "TOTAL_SALES", "TOTAL_UNITS", "TOTAL_COUPONS"]
sale_new_df = sale_only_df[sales_weather_cols].copy()

# rename column header
sale_new_df = sale_new_df.rename(columns={"DATE": "date"})
sale_new_df.head()

,date,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS
0,1/1/2016,S,1604,101482.59,8046,769
2,1/2/2016,S,2669,174602.55,13305,1446
4,1/3/2016,S,2298,152357.03,11795,1324
6,1/4/2016,S,1967,109492.91,9075,947
8,1/5/2016,S,1796,108041.47,8424,662


In [7]:
# write df to csv
sale_new_df.to_csv("../Resources/Cleaned_Sales_Data.csv", index=False)

In [8]:
# merge sales and weather
df_weather_data['date'] = df_weather_data['date'].astype('datetime64[ns]')
sale_new_df['date'] = sale_new_df['date'].astype('datetime64[ns]')

sale_weather_df = pd.merge(sale_new_df, df_weather_data, on="date")
sale_weather_df.head()

,date,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,max_temp,min_temp,avg_temp,precipitation,snow_fall,snow_depth
0,2016-01-01,S,1604,101482.59,8046,769,42,34,38.0,0.0,0.0,0.0
1,2016-01-02,S,2669,174602.55,13305,1446,40,32,36.0,0.0,0.0,0.0
2,2016-01-03,S,2298,152357.03,11795,1324,45,35,40.0,0.0,0.0,0.0
3,2016-01-04,S,1967,109492.91,9075,947,36,14,25.0,0.0,0.0,0.0
4,2016-01-05,S,1796,108041.47,8424,662,29,11,20.0,0.0,0.0,0.0


In [9]:
# write df to csv
sale_weather_df.to_csv("../Resources/Weather_Sales_Data.csv", index=False)

In [10]:
# set index
weather_2016 = df_weather_data.copy()
weather_2016.set_index("date", inplace=True)
sales_2016 = sale_new_df.copy()
sales_2016.set_index("date", inplace=True)
weather_sales_2016 = sale_weather_df.copy()
weather_sales_2016.set_index("date", inplace=True)

In [11]:
# create database connection (need to enter mysql password)
connection_string = "root:raynor1128@localhost/weather_data"
engine = create_engine(f'mysql://{connection_string}')

In [12]:
# confirm tables
engine.table_names()

['sales_2016', 'weather_2016', 'weather_sales_2016']

### Load DataFrames into SQL Database

In [13]:
# load dataframes into sql database
weather_2016.to_sql(name='weather_2016', con=engine, if_exists='append', index=True)
sales_2016.to_sql(name='sales_2016', con=engine, if_exists='append', index=True)
weather_sales_2016.to_sql(name='weather_sales_2016', con=engine, if_exists='append', index=True)